[src](http://rickyhan.com/jekyll/update/2017/09/04/simgan-captcha.html)

[paper](https://arxiv.org/pdf/1612.07828v1.pdf)

## S+U 
* simulated + unsupervised learning
* use a set of real images to learn a refiner $\tilde x = R_\theta (x)$
* $L_R(\theta) = \sum_i l_\text{real} (\theta; \tilde x_i, y) + \lambda l_\text{reg} (\theta; \tilde{x_i}, x_i)$
* now the discriminator does this
* $L_D (\phi) = - \sum_i log(D_\phi (\tilde x_i)) - \sum_j log(1 - D_\phi (y_j))$
* $l_\text{real} (\theta; \tilde{x_i}, Y) = -\sum_i log(1 - D_\phi(R_\theta (x_i)))$

### final refiner definition
* $L_R(\theta) = - \sum_i log(1 - D_\phi R_\theta (x_i))) + \lambda \| R_\theta(x_i) - x_i \|$
* $R_\theta$ is a conv network, this preserves the image structure in general

In [9]:
import requests
import threading
URL = "https://captcha.delorean.codes/u/rickyhan/challenge"
DIR = "/tmp/"
NUM_CHALLENGES = 20
lock = threading.Lock()

In [4]:
def download_file(url, fname):
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(fname, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    with lock:
        pass
        # print fname


ts = []
for i in range(NUM_CHALLENGES):
    fname = "/tmp/" + "challenge-{}".format(i)
    t = threading.Thread(target=download_file, args=(URL, fname))
    ts.append(t)
    t.start()
for t in ts:
    t.join()
print "Done"

Done


In [11]:
import json, base64, os
IMG_DIR = "/tmp/"
fnames = ["{}challenge-{}".format(DIR, i) for i in range(NUM_CHALLENGES)]
if not os.path.exists(IMG_DIR):
    os.mkdir(IMG_DIR)
def save_imgs(fname):
    with open(fname) as f:
        l = json.loads(f.read())

    for image in l['images']:
        b = base64.decodestring(image['jpg_base64'])
        name = image['name']
        with open(IMG_DIR+"/{}.jpg".format(name), 'w') as f:
            f.write(b)

for fname in fnames:
    save_imgs(fname)
assert len(os.listdir(IMG_DIR)) == 1000 * NUM_CHALLENGES

ValueError: No JSON object could be decoded